In [32]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [33]:
pd.options.display.max_columns = None

#### Leer archivo calendario para calcular las noches de ocupacion que tiene reservado cada hospedaje

In [34]:
df_calendar=pd.read_csv('../CLASE_11_Pandas2/calendar.csv')

In [35]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8661286 entries, 0 to 8661285
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 462.6+ MB


#### Convertir fecha a formato datetime
#### Corregir formato de precio eliminando el signos pesos y cambiando ',' por '' y '.' por ','

In [36]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'])

In [37]:
df_calendar["price"] = df_calendar["price"].apply(lambda x: x.replace("$",""))

In [38]:
df_calendar["adjusted_price"] = df_calendar["adjusted_price"].apply(lambda x: x.replace("$",""))

In [39]:
df_calendar["price"] = df_calendar["price"].apply(lambda x: x.replace(",",""))

In [40]:
df_calendar["adjusted_price"] = df_calendar["adjusted_price"].apply(lambda x: x.replace(",",""))

#### Crear columna que tenga una 1 si el dia esta ocupado, crear columna que diga que dia de la semana es y convertir en dummies para poder calcular ocupacion por dia de semana. Agregamos la clasificacion de dia de semana y dia habil. 

In [41]:
df_calendar['ocupado']=df_calendar['available'].apply(lambda x: 0 if x =='f' else 1)

In [43]:
df_calendar['dia_semana']=(df_calendar['date']).apply(lambda x: x.weekday())

In [44]:
df_calendar= pd.get_dummies(df_calendar,columns=['dia_semana'], drop_first = False, prefix='dia')

In [46]:
df_calendar["habil"] = df_calendar.apply(lambda x: 0 if x['dia_5']==1 or x['dia_6']==1 else 1, axis=1)

In [47]:
df_calendar["finde"] = df_calendar.apply(lambda x: 1 if np.logical_or( x['dia_5']==1, x['dia_6']==1) else 0, axis=1)

In [48]:
lista=['dia_0','dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','habil','finde']
for i in lista:
    df_calendar[i]=df_calendar[i]*df_calendar['ocupado']


#### Agrupar por listing_id para saber cuantas reservas hay por alojamiento en el periodo estudiado

In [50]:
df_ocupacion= df_calendar.groupby('listing_id').agg({'ocupado':'sum',
                                                     'dia_0':'sum',  
                                                     'dia_1':'sum',
                                                     'dia_2':'sum',
                                                     'dia_3':'sum',
                                                     'dia_4':'sum',
                                                     'dia_5':'sum',
                                                     'dia_6':'sum',
                                                     'finde':'sum',
                                                      'habil':'sum',
                                                      'adjusted_price':'max'})

In [93]:
df_ocupacion

,ocupado,dia_0,dia_1,dia_2,dia_3,dia_4,dia_5,dia_6,finde,habil,adjusted_price
listing_id,,,,,,,,,,,
11508,365,52,52,52,52,52,52,53,105,260,3983.00
12463,365,52,52,52,52,52,52,53,105,260,1792.00
13095,365,52,52,52,52,52,52,53,105,260,2987.00
13096,365,52,52,52,52,52,52,53,105,260,2987.00
13097,365,52,52,52,52,52,52,53,105,260,2987.00
...,...,...,...,...,...,...,...,...,...,...,...
43278316,365,52,52,52,52,52,52,53,105,260,5111.00
43280399,365,53,52,52,52,52,52,52,104,261,980.00
43280552,52,7,7,8,8,8,7,7,14,38,1860.00


### Leer archivo listings.csv 

In [56]:
df_listings=pd.read_csv('../CLASE_11_Pandas2/listings.csv')

C:\Users\silde\anaconda3\envs\dhdsblend2021\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (61,62,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
df_listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11508,https://www.airbnb.com/rooms/11508,20200426042522,2020-04-26,Amazing Luxurious Apt-Palermo Soho,NaN,LUXURIOUS NEW APT: 1 BDRM- POOL/ GYM/ SPA/ 24-...,LUXURIOUS NEW APT: 1 BDRM- POOL/ GYM/ SPA/ 24-...,none,AREA: PALERMO SOHO Minutes walking distance fr...,NaN,All major bus lines are nearby. Stop: Plaza It...,Outdoor pool and outdoor area on first floor F...,NaN,"During your stay in the apt, you must follow t...",NaN,NaN,https://a0.muscache.com/im/pictures/19357696/b...,NaN,42762,https://www.airbnb.com/users/show/42762,Candela,2009-10-01,"New York, New York, United States",-,within a few hours,100%,100%,t,https://a0.muscache.com/im/users/42762/profile...,https://a0.muscache.com/im/users/42762/profile...,Palermo,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Buenos Aires, Capital Federal, Argentina",Palermo,Palermo,NaN,Buenos Aires,Capital Federal,NaN,Buenos Aires,"Buenos Aires, Argentina",AR,Argentina,-34.58184,-58.42415,t,Condominium,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,"$3,983.00","$39,828.00","$82,975.00",$0.00,"$3,319.00",1,$0.00,2,1125,2,2,1125,1125,2.0,1125.0,2 months ago,t,30,60,90,365,2020-04-26,26,4,2012-07-02,2019-11-28,95.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.27
1,12463,https://www.airbnb.com/rooms/12463,20200426042522,2020-04-26,Room in Recoleta - awesome location,My apartment is centrally located in Recoleta ...,This is a very comfortable pull-out sofa in th...,My apartment is centrally located in Recoleta ...,none,It's near the school of medicine so the street...,NaN,Just a block from the Facultad de Medicina sub...,"Wi-Fi, kitchen, TV, Playstation 3",I work out of my apartment so I am around duri...,"Nothing formal, but no smoking and no guests w...",NaN,NaN,https://a0.muscache.com/im/pictures/11680256/5...,NaN,48799,https://www.airbnb.com/users/show/48799,Matt,2009-10-28,"Danbury, Connecticut, United States",I'm an expatriate from the USA who has been li...,within a few hours,100%,NaN,f,https://a0.muscache.com/im/pictures/user/2e5c6...,https://a0.muscache.com/im/pictures/user/2e5c6...,Recoleta,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Buenos Aires, Capital Federal, 

#### crear dataset corrdenadas que tenga un valor de latitud y longitud para cada barrio

In [61]:
df_coordenadas_b = df_listings[['id','neighbourhood_cleansed','latitude','longitude']]

In [67]:
df_coordenadas_b.rename(columns={'id': 'listing_id', 
                                 'latitude':'latitude_b',
                                 'longitude':'longitude_b'},
               inplace=True)

C:\Users\silde\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [68]:
df_coordenadas_g=df_coordenadas_b.groupby('neighbourhood_cleansed')['listing_id'].max()

In [69]:
df_coordenadas_g=pd.DataFrame(df_coordenadas_g)

In [ ]:
df_coordenadas_final = df_coordenadas_g.merge(df_coordenadas_b,on='listing_id',how='left')

In [ ]:
df_coordenadas_final.to_csv('coordendas')

#### crear un dataset con  conservando las columnas de df_listings relevantes para el analisis 

In [ ]:
df_hospedajes= df_listings[['id','name','host_location','host_response_time','host_response_rate','host_acceptance_rate','host_neighbourhood','neighbourhood','neighbourhood_cleansed','city','state','market','country_code','country','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','amenities','square_feet','price','weekly_price','monthly_price','extra_people','minimum_nights','maximum_nights','has_availability','availability_30','availability_60','availability_90','availability_365','number_of_reviews','review_scores_rating','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','reviews_per_month']]

In [73]:
df_hospedajes.rename( columns = { "id": "listing_id"}, inplace=True)

C:\Users\silde\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
df_hospedajes.info()

In [ ]:
df_hospedajes.columns

In [ ]:
df_hospedajes['country'].unique()

In [75]:
df_hospedajes_arg=df_hospedajes[df_hospedajes.country=='Argentina']

In [ ]:
df_hospedajes_arg.info()

In [76]:
df_ocupacion.reset_index()

,listing_id,ocupado,dia_0,dia_1,dia_2,dia_3,dia_4,dia_5,dia_6,finde,habil,adjusted_price
0,11508,365,52,52,52,52,52,52,53,105,260,3983.00
1,12463,365,52,52,52,52,52,52,53,105,260,1792.00
2,13095,365,52,52,52,52,52,52,53,105,260,2987.00
3,13096,365,52,52,52,52,52,52,53,105,260,2987.00
4,13097,365,52,52,52,52,52,52,53,105,260,2987.00
...,...,...,...,...,...,...,...,...,...,...,...,...
23724,43278316,365,52,52,52,52,52,52,53,105,260,5111.00
23725,43280399,365,53,52,52,52,52,52,52,104,261,980.00
23726,43280552,52,7,7,8,8,8,7,7,14,38,1860.00
23727,43281112,178,26,26,26,26,25,24,25,49,129,2861.00


In [77]:
df_hospedajes_arg.columns

Index(['listing_id', 'name', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_neighbourhood',
       'neighbourhood', 'neighbourhood_cleansed', 'city', 'state', 'market',
       'country_code', 'country', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price',
       'extra_people', 'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')

#### Unir la informacion de ocupacion extraida del df_calendario con la informacion de listings y obtener el archivo de salida para realizar el analisis en PBI

In [78]:
 df_final=df_ocupacion.merge(df_hospedajes_arg,on='listing_id',how='left')

In [79]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23729 entries, 0 to 23728
Data columns (total 52 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_id                                    23729 non-null  int64  
 1   ocupado                                       23729 non-null  int64  
 2   dia_0                                         23729 non-null  int64  
 3   dia_1                                         23729 non-null  int64  
 4   dia_2                                         23729 non-null  int64  
 5   dia_3                                         23729 non-null  int64  
 6   dia_4                                         23729 non-null  int64  
 7   dia_5                                         23729 non-null  int64  
 8   dia_6                                         23729 non-null  int64  
 9   finde                                         23729 non-null 

In [80]:
df_final['adjusted_price']=df_final['adjusted_price'].astype(float)

In [81]:
df_final.describe()

,listing_id,ocupado,dia_0,dia_1,dia_2,dia_3,dia_4,dia_5,dia_6,finde,habil,adjusted_price,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,2.372900e+04,23729.00000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23728.000000,23728.000000,23728.000000,23666.000000,23685.000000,23562.000000,383.000000,23728.000000,23728.000000,23728.000000,23728.000000,23728.000000,23728.000000,23728.000000,16839.000000,23728.000000,23728.000000,23728.00000,17222.000000
mean,2.668555e+07,212.02832,30.308652,30.317670,30.355261,30.328627,30.218003,30.101058,30.399048,60.500105,151.528214,3850.988664,-34.592574,-58.415902,2.761548,1.259212,1.135909,1.863085,486.227154,6.027478,649.555335,19.494521,41.350851,63.908378,209.382291,16.314017,94.652830,5.401719,0.551585,0.09415,1.053696
std,1.263388e+07,136.90341,19.550473,19.491564,19.492662,19.471469,19.517824,19.649293,19.792156,39.436253,97.489199,18823.904660,0.018282,0.029925,1.480245,0.631624,0.858939,1.486757,868.125617,25.635994,879.457470,12.985377,24.821373,35.917006,137.994098,32.322767,8.892571,14.421867,1.718835,0.89358,1.158269
min,1.150800e+04,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.000000,-34.688950,-58.530200,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.00000,0.010000
25%,1.765680e+07,89.00000,13.000000,13.000000,13.000000,13.000000,13.000000,12.000000,12.000000,24.000000,65.000000,1400.000000,-34.603430,-58.435380,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,60.000000,0.000000,19.000000,36.000000,89.000000,0.000000,93.000000,1.000000,0.000000,0.00000,0.230000
50%,3.000878e+07,180.00000,26.000000,26.000000,26.000000,26.000000,26.000000,25.000000,26.000000,51.000000,129.000000,2191.000000,-34.591890,-58.415165,2.000000,1.000000,1.000000,1.000000,377.000000,3.000000,1125.000000,28.000000,58.000000,87.000000,180.000000,4.000000,97.000000,1.000000,0.000000,0.00000,0.640000
75%,3.736241e+07,363.00000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,104.000000,259.000000,3319.000000,-34.582150,-58.391757,4.000000,1.500000,1.000000,2.000000,646.000000,4.000000,1125.000000,30.000000,60.000000,90.000000,363.000000,18.000000,100.000000,2.000000,0.000000,0.00000,1.470000
max,4.328606e+07,366.00000,53.000000,52.000000,52.000000,52.000000,52.000000,52.000000,53.000000,105.000000,261.000000,923609.000000,-34.534100,-58.355410,16.000000,30.000000,30.000000,50.000000,13907.000000,1125.000000,99999.000000,30.000000,60.000000,90.000000,365.000000,500.000000,100.000000,99.000000,26.000000,19.00000,10.730000


#### crear una columna outliers con los valores de precio con el criterio de los 3 sigma

In [82]:
avg= df_final['adjusted_price'].mean()

In [83]:
sigma= df_final['adjusted_price'].std()

In [84]:
tope_s= avg + 3*sigma
tope_i=avg - 3*sigma

In [85]:
df_final['outliers']=df_final.apply(lambda x: 0 if tope_i<x['adjusted_price']<tope_s else 1, axis=1)

In [90]:
df_final.to_csv('hospedajes_final')